In [489]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

In [490]:
external_stylesheets = ['assets/style.css', dbc.themes.BOOTSTRAP]
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)
style_card = {"width": "18rem", 'margin': '20px', 'display': 'inline-block', 'vertical-align': 'top', 'text-align': 'center', 'padding': '10px', 'background-color': '#ffffff', 'color': 'black', 'font-family': 'Arial, sans-serif'}
# define a card component
card1 = dbc.Card(
    [
        dbc.CardImg(src="./assets/batting.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("BatsMen", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/batting"),
            ]
        ),
    ],
    style=style_card,
)

card2 = dbc.Card(
    [
        dbc.CardImg(src="./assets/bowling.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Bowler", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/bowling"),
            ]
        ),
    ],
    style=style_card,
)

card3 = dbc.Card(
    [
        dbc.CardImg(src="./assets/stadium.jpg", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Stadiums", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/stadium"),
            ]
        ),
    ],
    style=style_card,
)

card4 = dbc.Card(
    [
        dbc.CardImg(src="./assets/ranking.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Ranking", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/rank"),
            ]
        ),
    ],
    style=style_card,
)

card5 = dbc.Card(
    [
        dbc.CardImg(src="./assets/team.png", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Teams", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/team"),
            ]
        ),
    ],
    style=style_card,
)

card6 = dbc.Card(
    [
        dbc.CardImg(src="./assets/summary.jpg", top=True, style={'height':'16rem'}),
        dbc.CardBody(
            [
                html.H4("Match Summary", className="card-title"),
                html.P(
                    "Some quick example text to build on the card.",
                    className="card-text",
                ),
                dbc.Button("Details", color="primary", href="/summary"),
            ]
        ),
    ],
    style=style_card,
)


# Define the layout for the home page
home_layout = html.Div(children=[
    html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
    html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
]),
    card1, card2, card3, card4, card5, card6
    
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})



### Batsmen Layout

In [491]:
run_per_batsman_per_season = pd.read_csv('Total_run_per_batsman_per_Season.csv')
batter = sorted(run_per_batsman_per_season['batter'].unique())
top_batsmen_per_season = run_per_batsman_per_season.groupby(['Season']).max().reset_index()

batsmen_layout  = html.Div(children=[
    html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
    html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),]),
    html.A(html.Button('Go back to home', style={'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}), href='/'),
    html.Div([
        html.H1("Total runs per season", style={'color': 'black', 'font-family': 'Arial, sans-serif'}),
        dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': player, 'value': player} for player in batter],
            value=batter[0]
        ),
        dcc.Graph(id='line-chart-batsman')
        ]),
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})

@app.callback(
    Output('line-chart-batsman', 'figure'),
    [Input('player-dropdown', 'value')]
)
def update_line_chart1(selected_player):
    # Filter the dataframe based on the selected team
    filtered_df = run_per_batsman_per_season[run_per_batsman_per_season['batter'] == selected_player]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='batsman_run', title=f'Total runs per season for {selected_player}')
    return fig

In [492]:
num_wicket_per_bowler = pd.read_csv('./num_wickets_per_bowler.csv')
bowler = sorted(num_wicket_per_bowler['bowler'].unique())


bowler_layout  = html.Div(children=[
    html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
    html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),]),
    html.A(html.Button('Go back to home', style={'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}), href='/'),
    html.Div([
        html.H1("Total wickets per season", style={'color': 'black', 'font-family': 'Arial, sans-serif'}),
        dcc.Dropdown(
            id='player-dropdown-bowler',
            options=[{'label': player, 'value': player} for player in bowler],
            value=bowler[0]
        ),
        dcc.Graph(id='line-chart-bowler')
        ]),
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})

@app.callback(
    Output('line-chart-bowler', 'figure'),
    [Input('player-dropdown-bowler', 'value')]
)
def update_line_chart2(selected_player):
    # Filter the dataframe based on the selected team
    filtered_df = num_wicket_per_bowler[num_wicket_per_bowler['bowler'] == selected_player]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='Wickets', title=f'Total wickets per season for {selected_player}')
    return fig


In [493]:

matches_per_city = pd.read_csv('./num_matches_per_city.csv')
# data = data.sort_values(['Season', 'Count'])

season_options = [{'label': str(season), 'value': season} for season in matches_per_city['Season'].unique()]

#define layout
stadium_layout = html.Div(children=[
        html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
        html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),]),
        html.A(html.Button('Go back to home', style={'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}), href='/'),
        html.Div(children=[
        html.H1(children='Number of Matches Played in Each City'),
        dcc.Dropdown(
            id='season-dropdown-2',
            options=season_options,
            value=matches_per_city['Season'].unique()[0]
        ),
        dcc.Graph(
            id='matches-per-city',
            figure={}
        )
    ])
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})




# Define the callback function
@app.callback(
    dash.dependencies.Output('matches-per-city', 'figure'),
    [dash.dependencies.Input('season-dropdown-2', 'value')]
)
def update_figure(selected_season):
    # Filter the data based on the selected season
    filtered_data = matches_per_city[matches_per_city['Season'] == selected_season].sort_values('Count')
    
    # Compute the count of matches in each city

    # Create the figure
    fig = px.bar(filtered_data, x='Count', y='City', orientation='h', 
                 title=f'Number of Matches Played in Each City in {selected_season}')
    
    return fig


In [494]:
rank_data = pd.read_csv('./Rank_data.csv')
rank_data_fig = px.line(rank_data, x='Season', y='Rank', color='Team', title='Rank of each team over the years')
rank_data_fig.update_layout(yaxis=dict(autorange='reversed'), yaxis2=dict(autorange='reversed', overlaying='y', side='right'))
# options contains team name
options = [{'label': team, 'value': team} for team in rank_data['Team'].unique()]
# Create layout
ranking_layout = html.Div(children=[
    html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
        html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),]),
        html.A(html.Button('Go back to home', style={'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}), href='/'),
        html.Div(children=[
        html.H1(children='Rank Chart'),
        dcc.Dropdown(
            id='multi-select-dropdown',
            options=options,
            value=[],
            multi=True
        ),
        dcc.Graph(
            id='bump-chart'
        )
    ])
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})

@app.callback(
    dash.dependencies.Output('bump-chart', 'figure'),
    dash.dependencies.Input('multi-select-dropdown', 'value'))
def update_bump_chart(selected_values):
    # update the figure based on the selected values
    rank_data_fig = px.line(rank_data[rank_data['Team'].isin(selected_values)], x='Season', y='Rank', color='Team', title='Rank of each team over the years')
    rank_data_fig.update_layout(yaxis=dict(autorange='reversed'), yaxis2=dict(autorange='reversed', overlaying = 'y', side = 'right'))
    return rank_data_fig



In [495]:
total_run_per_team = pd.read_csv('Total_run_per_team_per_Season.csv')

# Create a list of teams for the dropdown
teams = sorted(total_run_per_team['BattingTeam'].unique())

# Define the layout of the app
team_layout = html.Div(children=[
    html.Div(style={'max-width':'1400px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#223577', 'padding': '30px'},children=[
    html.H1('Indian Premier League', style={'color': 'white', 'font-family': 'Arial, sans-serif'}),]),
    html.A(html.Button('Go back to home', style={'margin': '20px', 'background-color': '#223577', 'color': 'white', 'font-family': 'Arial, sans-serif', 'cursor':'pointer', 'font-size':'16px', 'border-radius':'7px'}), href='/'),
        html.Div([
        html.H1("Total runs per season for each team"),
        dcc.Dropdown(
            id='team-dropdown-team',
            options=[{'label': team, 'value': team} for team in teams],
            value=teams[0]
        ),
        dcc.Graph(id='line-chart-team')
    ])
], style={'max-width':'1400px', 'text-align': 'center', 'padding':'30px', 'margin': '0 auto'})




# Define the callback function to update the line chart based on the selected team
@app.callback(
    Output('line-chart-team', 'figure'),
    [Input('team-dropdown-team', 'value')]
)
def update_line_chart3(selected_team):
    # Filter the dataframe based on the selected team
    filtered_df = total_run_per_team[total_run_per_team['BattingTeam'] == selected_team]
    filtered_df.sort_values('Season', inplace=True)
    # Create a line chart using Plotly Express
    fig = px.line(filtered_df, x='Season', y='total_run', title=f'Total runs per season for {selected_team}')
    return fig

In [496]:

# app = dash.Dash(__name__)
df = pd.read_csv('./match_summary_runs.csv')
summary_layout = html.Div([
    dcc.Dropdown(
        id='year-dropdown-summary',
        options=[{'label': year, 'value': year} for year in df['Season'].unique()],
        value=df['Season'].unique()[0],
        clearable=False
    ),
    dcc.Dropdown(
        id='team1-dropdown-summary',
        options=[],
        value='',
        clearable=False
    ),
    dcc.Dropdown(
        id='team2-dropdown-summary',
        options=[],
        value='',
        clearable=False
    ),
    dcc.Dropdown(
        id='match-dropdown-summary',
        options=[],
        value='',
        clearable=False
    ),
    dcc.Graph(id='bar-plot-summary')
])
@app.callback(
    Output('team-dropdown-summary', 'options'),
    [Input('year-dropdown-summary', 'value')]
)
def update_team_dropdown_summary(year):
    teams = df[df['Season'] == year]['BattingTeam'].unique()
    options = [{'label': team, 'value': team} for team in teams]
    return options


@app.callback(
    Output('match-dropdown-summary', 'options'),
    [Input('year-dropdown-summary', 'value'),
     Input('team1-dropdown-summary', 'value'),
     Input('team2-dropdown-summary', 'value')]
)
def update_match_dropdown_summary(year, team1, team2):
    matches = df[(df['Season'] == year) & (df['BattingTeam'] == team1) & (df['BattingTeam']==team2)]['ID'].unique()
    options = [{'label': match, 'value': match} for match in matches]
    return options


@app.callback(
    Output('bar-plot-summary', 'figure'),
    [Input('year-dropdown-summary', 'value'),
     Input('team-dropdown-summary', 'value'),
     Input('match-dropdown-summary', 'value')]
)
def update_bar_plot_summary(year, team, match):
    data = df[(df['Season'] == year) & (df['BattingTeam'] == team) & (df['ID'] == match)]
    fig = px.bar(data, x='overs', y='total_run', color='BattingTeam',
                 title=f'Run per over for {team} in match {match}', labels={'total_run': 'Run/Over'})
    fig.update_layout(
        xaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=1
        )
    )
    return fig




In [497]:

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/batting':
        return batsmen_layout
    elif pathname == '/bowling':
        return bowler_layout
    elif pathname == '/stadium':
        return stadium_layout
    elif pathname == '/rank':
        return ranking_layout
    elif pathname == '/team':
        return team_layout
    elif pathname == '/summary':
        return summary_layout
    else:
        return home_layout

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
